Numeric LSTM
=====

Essa é a implementação de uma LSTM para ser treinada com operações aritméticas (a princípio, soma e subtração). Números são lidos um dígito de cada vez, de modo que a LSTM aprenda uma representação interna para quantidades. Espera-se que o aprendizado sobre como representar quantidades seja alcançado através do treinamento para operações aritméticas.

Funcionamento geral
----

1. LSTM encoder codifica dois números

2. Camada linear (ou relu) aplicada sobre a concatenação dos dois números para fazer a soma sobre a representação codificada. Deveria haver um conjunto de pesos para cada operação suportada

3. LSTM decoder gera a saída, um algarismo de cada vez

Obs:

- A mesma LSTM é usada para codificar os dois números de entrada, sendo resetada após o primeiro

- Embeddings são compartilhadas, mas o encoder e decoder têm seus próprios parâmetros

Ideias relevantes para avaliação
----

O objetivo aqui é a geração de embeddings para números arbitrários. É desejável ter uma avaliação que mostre se essas embeddings realmente são úteis quando combinadas com um modelo de espaço vetorial para um vocabulário normal. Algumas ideias:

- Gerar artificialmente várias sentenças com contruções do tipo "há 20 pessoas", "há mais de 10 pessoas" e tentar fazer algum tipo de classificação, como RTE

- Avaliar essas embeddings em algum corpus de RTE onde haja muitos números, e contrastar com outras variantes que não tenham as embeddings numéricas. Um problema seria com o que comparar...

In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
# random seed
random_seed = 42
random.seed(random_seed)

class Symbol(object):
    """
    Placeholder class for values used in the RNNs.
    """
    END = 10
    GO = 11

Geração de batches
---

In [35]:
def int_to_array(x):
    """
    This is slower than array to int
    """
    if x == 0:
        return np.array([0])
    
    digits = []
    while x > 0:
        digits.insert(0, x % 10)
        x /= 10
    
    return np.array(digits)


def batch_array_to_ints(x):
    """
    Return an array containing the integers represented by each
    row of the input array.
    """
    total = np.zeros_like(x[0])
    for row in x:
        total = 10 * total + row
    return total


def array_to_int(x):
    total = 0
    for i in x:
        total = 10 * total + i
    return total


def ints_to_array_batch(x, max_time_steps):
    """
    Takes an integer 1-D array and returns a 2-D array with their digits.
    
    The most significant figures are at the index 0
    
    :param x: 1-D integer array
    :param max_time_steps: the maximum possible number of time steps
    :return: 2-D array with shape (max_time_steps, len(x)) and one digit per cell.
    """
    x = np.copy(x)
    max_power_10 = max_time_steps - 2
    
    # since the last row of the resulting array must be the END symbol,
    # this is the maximum number allowed
    max_possible_number = 10 ** (max_time_steps - 1) - 1
    assert not any(x > max_possible_number), 'Cannot represent a number in the array'
    
    new = np.full((max_time_steps, len(x)), Symbol.END, np.int32)
    # just to make sure in case of the results is 0
    new[0] = 0
    
    # start from 2 so that the last row always has END
    for i in range(2, max_time_steps + 1):
        big_ones = x >= (10 ** max_power_10)
        new[-i][big_ones] = x[big_ones] % 10
        x[big_ones] /= 10
        
        # on a related note: raising a variable to a power is faster than 
        # dividing it by 10
        max_power_10 -= 1
    
    return new


def ints_to_array_batch_reversed(x):
    x = np.copy(x)
    new = np.full((8, len(x)), 10, np.int32)
    # just to make sure in case the result is 0
    new[0] = 0

    for i in range(8):
        new[i][x > 0] = x[x > 0] % 10
        x /= 10
    
    return new


def generate_numbers(num_time_steps, num_padding, batch_size):
    """
    Generate a 2-D array containing digits, and a corresponding
    array with their concatenation.
    
    Each number will have the determined number of valid time steps,
    plus the given number of padding (as the END symbol)
    """
    shape = (num_time_steps, batch_size)
    digits = np.random.random_integers(0, 9, shape)
    numbers = batch_array_to_ints(digits)
    padding = np.full((num_padding, batch_size), 
                      Symbol.END, np.int32)
    digits = np.concatenate([digits, padding])
    
    return (digits, numbers)

In [36]:
def generate_data():
    digits_pool = []
    number_pool = []
    
    for i in range(1, input_sequence_size):
        # we generate 10 1-digit numbers, 20 2-digit, 30 3-digit etc
        num_padding = input_sequence_size - i
        digits, numbers = generate_numbers(i, num_padding, i * 10)
        digits_pool.append(digits)
        number_pool.append(numbers)
    
    all_digits = np.concatenate(digits_pool, 1)
    all_numbers = np.concatenate(number_pool)
    
    # shuffle both lists with the same permutation
    rng_state = np.random.get_state()
    # shuffle works row-wise, but we want to shuffle columns
    np.random.shuffle(all_digits.T)
    np.random.set_state(rng_state)
    np.random.shuffle(all_numbers)
    
    # now divide it in the middle to get the two terms
    middle = all_digits.shape[1] / 2
    first_terms = all_digits[:, :middle]
    first_terms_values = all_numbers[:middle]
    second_terms = all_digits[:, middle:]
    second_terms_values = all_numbers[middle:]
    
    results_integer = first_terms_values + second_terms_values
    results = ints_to_array_batch(results_integer, output_sequence_size)
    first_sizes = np.sum(first_terms != 10, 0)
    second_sizes = np.sum(second_terms != 10, 0)
    
    return (first_terms, second_terms, 
            first_sizes, second_sizes,
            results)

Criação do grafo
---

**Código abaixo reseta o grafo**

In [165]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [166]:
embedding_size = 50
num_lstm_units = 100

# a number with 8 digits at most
input_sequence_size = 8

# a number with 9 digits at most plus at least one END symbol
output_sequence_size = 10

# digits 0-9, GO and END symbols
vocab_size = 12

graph = tf.Graph()

# both terms are inputs to the encoder
first_term = tf.placeholder(tf.int32, [input_sequence_size, None], 'first_term')
second_term = tf.placeholder(tf.int32, [input_sequence_size, None], 'second_term')
result_data = tf.placeholder(tf.int32, [output_sequence_size, None], 'result')

first_term_size = tf.placeholder(tf.int32, [None], 'first_term_size')
second_term_size = tf.placeholder(tf.int32, [None], 'second_term_size')

l2_constant = tf.placeholder(tf.float32, 1, name='l2_constant')

# we want to share the embeddings between encoder and decoder, but not all parameters
shape = [vocab_size, embedding_size]
embeddings = tf.Variable(tf.random_uniform(shape, -1.0, 1.0), name='embeddings')

lstm_initializer = tf.random_uniform_initializer(-0.1, 0.1)
lstm_cell = tf.nn.rnn_cell.LSTMCell(num_lstm_units, embedding_size, 
                                    initializer=lstm_initializer)


def generate_rnn_input(sequence_indices, num_time_steps):
    """
    Generate the input to the RNN from a tensor of shape [sequence_size, batch_size]
    Return a list of tensors of shape [batch_size, embedding_size]
    """
    embedded_sequence =  tf.nn.embedding_lookup(embeddings, sequence_indices)
    return [tf.squeeze(time_step, [0]) 
            for time_step in tf.split(0, num_time_steps, embedded_sequence)]
    
input_1st_term = generate_rnn_input(first_term, input_sequence_size)
input_2nd_term = generate_rnn_input(second_term, input_sequence_size)

# create a tensor with the GO embedding with batch size
embedded_go = tf.nn.embedding_lookup(embeddings, Symbol.GO)
batch_go = tf.ones_like(input_1st_term[0]) * embedded_go

result_labels = [tf.squeeze(time_step, [0]) 
                 for time_step in tf.split(0, output_sequence_size, result_data)]
embedded_results = generate_rnn_input(result_data, output_sequence_size)
decoder_input = [batch_go] + embedded_results[:-1]

with tf.variable_scope('encoder') as encoder_scope:
    _, state_1st_term = tf.nn.rnn(lstm_cell, input_1st_term, 
                                  sequence_length=first_term_size, dtype=tf.float32)
    encoder_scope.reuse_variables()
    _, state_2nd_term = tf.nn.rnn(lstm_cell, input_2nd_term, 
                                  sequence_length=second_term_size, dtype=tf.float32)

two_terms = tf.concat(1, [state_1st_term, state_2nd_term])
    
with tf.variable_scope('addition_layer') as addition_scope:
    # the addition layer has as input a concatenation of two encoded arrays
    # its output is the input for the decoder LSTM
    shape = [2 * lstm_cell.state_size, lstm_cell.state_size]
    addition_weights = tf.Variable(tf.truncated_normal(shape, 0.0, 0.1))
    addition_bias = tf.Variable(tf.zeros([lstm_cell.state_size]))
    
addition_output = tf.nn.xw_plus_b(two_terms, addition_weights, addition_bias)

with tf.variable_scope('decoder') as decoder_scope:
    decoder_outputs, _ = tf.nn.seq2seq.rnn_decoder(decoder_input, addition_output, lstm_cell)
    
with tf.variable_scope('output_softmax') as softmax_scope:
    # softmax to map decoder raw output to digits
    shape = [num_lstm_units, vocab_size]
    softmax_weights = tf.Variable(tf.truncated_normal(shape, 0.0, 0.1))
    softmax_bias = tf.Variable(tf.zeros([vocab_size]))    

def project_output(raw_outputs, return_softmax=False):
    """
    Multiply the raw_outputs by a weight matrix, add a bias and return the
    softmax distribution or the logits.
    
    :param return_softmax: if True, return the softmaxes. If False, return
        the logits
    """
    output_logits = [tf.nn.xw_plus_b(time_step, softmax_weights, softmax_bias)
                     for time_step in raw_outputs]
    if not return_softmax:
        return output_logits
    
    output_softmax = [tf.nn.softmax(time_step) for time_step in output_logits]
    return output_softmax

# label_weights is just used to weight the importance of each class
label_weights = [tf.ones_like(result_labels[0], dtype=tf.float32)
                 for _ in result_labels]

output_logits = project_output(decoder_outputs, False)
l2_loss = l2_constant * (tf.nn.l2_loss(softmax_weights) + tf.nn.l2_loss(addition_weights))
loss = tf.nn.seq2seq.sequence_loss(output_logits, result_labels, label_weights) + l2_loss

Optimizer
---

In [167]:
starting_learning_rate = 0.2

global_step = tf.Variable(0, trainable=False)
#learning_rate = tf.train.exponential_decay(starting_learning_rate, global_step, 10000, 0.98)

optimizer = tf.train.AdamOptimizer(starting_learning_rate, epsilon=0.1)
gradients, v = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(gradients, 1.25)

train_op = optimizer.apply_gradients(zip(gradients, v), 
                                     global_step=global_step)

Execução de novos inputs
---

Roda o encoder, a soma e o decoder por um passo

In [169]:
# we use the same intermediate results of the training part until the addition
# the difference is that the decoder must use its own output

digit_step = tf.placeholder(tf.int32, [None], 'digit_step')
decoder_step_state = tf.placeholder(tf.float32, [None, lstm_cell.state_size], 
                                    'decoder_step_state')

# embed the input digit
decoder_step_input = [tf.nn.embedding_lookup(embeddings, digit_step)]


with tf.variable_scope(decoder_scope) as exec_time_decoder:
    exec_time_decoder.reuse_variables()
    decoder_step_output, decoder_new_state = tf.nn.seq2seq.rnn_decoder(decoder_step_input, 
                                                                       decoder_step_state, lstm_cell)
    
    step_logits = tf.nn.xw_plus_b(decoder_step_output[0], softmax_weights, softmax_bias)
    next_symbol = tf.argmax(step_logits, 1)

In [184]:
n = 2
sample_first_term = batch[0][:, n].reshape((-1, 1))
sample_second_term = batch[1][:, n].reshape((-1, 1))
sample_first_size =  batch[2][n].reshape((1))
sample_second_size =  batch[3][n].reshape((1))
sample_result = batch[4][:, n].reshape((-1, 1))

In [185]:
feeds = {first_term: sample_first_term,
         second_term: sample_second_term,
         first_term_size: sample_first_size,
         second_term_size: sample_second_size}
sample_decoder_state = sess.run(addition_output, feed_dict=feeds)

answer = []
current_symbol = [Symbol.GO]
while True:
    decoder_feeds = {decoder_step_state: sample_decoder_state,
                     digit_step: current_symbol}
    
    fetches = sess.run([next_symbol, decoder_new_state], 
                       feed_dict=decoder_feeds)
    current_symbol, sample_decoder_state = fetches
    
    if current_symbol == Symbol.END:
        break
    
    answer.append(current_symbol)

In [186]:
print(''.join(str(x[0]) for x in answer))
print(''.join(str(x) for x in sample_result.reshape(-1)))

45057
449571010101010


Treinamento
----

In [168]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(0, 20000):
    batch = generate_data()
    feeds = {first_term: batch[0], 
             second_term: batch[1],
             first_term_size: batch[2],
             second_term_size: batch[3],
             result_data: batch[4],
             l2_constant: [1e-5]}
    
    try:
        _, loss_value = sess.run([train_op, loss], feed_dict=feeds)
    except:
        add = addition_output.eval(feed_dict=feeds, session=sess)
        print(add)
        print(add.shape)
        raise
    
    if (i + 1) % 100 == 0:
        print('Epoch %d' % (i + 1))
        print('Loss: %.5f' % loss_value)

# sample_1st_term = np.random.random_integers(0, 9, (20, 1))
# test_feeds = {sample_input: sample_sequence, sample_size: [6]}
# np_encoded_state = sess.run([sample_encoder_state], feed_dict=test_feeds)[0]

# chosen_digit = [Symbol.GO]
# answer = []

# while len(answer) < max_sample_size:
#     decoder_feeds = {sample_decoder_input: [chosen_digit],
#                      sample_decoder_state: np_encoded_state}
#     np_softmax, np_encoded_state = sess.run([sample_softmax[0], new_decoder_state],
#                                              feed_dict=decoder_feeds)
#     chosen_digit = np_softmax.argmax(1)
    
#     if chosen_digit == Symbol.END:
#         break
#     else:
#         answer.append(chosen_digit)
    

#sess.close()

Epoch 100
Loss: 1.50703
Epoch 200
Loss: 1.41706
Epoch 300
Loss: 1.36825
Epoch 400
Loss: 1.35879
Epoch 500
Loss: 1.31487
Epoch 600
Loss: 1.27562
Epoch 700
Loss: 1.24197
Epoch 800
Loss: 1.19104
Epoch 900
Loss: 1.16172
Epoch 1000
Loss: 1.12861
Epoch 1100
Loss: 1.11705
Epoch 1200
Loss: 1.09908
Epoch 1300
Loss: 1.08800
Epoch 1400
Loss: 1.04927
Epoch 1500
Loss: 1.02802
Epoch 1600
Loss: 1.03931
Epoch 1700
Loss: 1.03907
Epoch 1800
Loss: 1.03325
Epoch 1900
Loss: 0.99064
Epoch 2000
Loss: 0.98467
Epoch 2100
Loss: 0.97119
Epoch 2200
Loss: 0.97819
Epoch 2300
Loss: 0.97482
Epoch 2400
Loss: 0.96442
Epoch 2500
Loss: 0.94674
Epoch 2600
Loss: 0.96394
Epoch 2700
Loss: 0.95011
Epoch 2800
Loss: 0.92620
Epoch 2900
Loss: 0.94196
Epoch 3000
Loss: 0.95102
Epoch 3100
Loss: 0.93077
Epoch 3200
Loss: 0.91106
Epoch 3300
Loss: 0.90492
Epoch 3400
Loss: 0.92678
Epoch 3500
Loss: 0.91659
Epoch 3600
Loss: 0.90096
Epoch 3700
Loss: 0.88646
Epoch 3800
Loss: 0.90194
Epoch 3900
Loss: 0.90068
Epoch 4000
Loss: 0.87564
Epoch 410